In [47]:
import xarray
import dask
import netCDF4
from pyhdf.SD import SD
import numpy
import pystare

In [48]:
file_path = '/home/griessbaum/MOD09/MOD09.A2019317.0815.006.2019319020759.hdf'
hdf = SD(file_path)
lon = hdf.select('Longitude').get().astype(numpy.double)
lat = hdf.select('Latitude').get().astype(numpy.double)

In [49]:
lat_x = xarray.DataArray(lat, dims=['x', 'y']).chunk({'x': 500})
lon_x = xarray.DataArray(lon, dims=['x', 'y']).chunk({'x': 500})

with dask.distributed.Client(n_workers=4) as client:
    s_d = xarray.apply_ufunc(pystare.from_latlon2D, 
                             lat_x,
                             lon_x,
                             dask='parallelized',
                             output_dtypes=[numpy.int64])

sids = numpy.array(s_d)

# Write NetCDF

In [54]:
i_1km = lon.shape[0]
j_1km = lon.shape[1]
l_1km = i_1km * j_1km # this is wrong!

sidecar = '/home/griessbaum/MOD09/MOD09.A2019317.0815.006.2019319020759_stare.hdf'

with netCDF4.Dataset(sidecar, "w", format="NETCDF4") as rootgrp:
    pass
    
with netCDF4.Dataset(sidecar, "a", format="NETCDF4") as rootgrp:
    rootgrp.createDimension("i_1km", i_1km)
    rootgrp.createDimension("j_1km", j_1km)
    rootgrp.createDimension("l_1km", i_1km*j_1km)

with netCDF4.Dataset(sidecar, "a", format="NETCDF4") as rootgrp:
    lats_netcdf = rootgrp.createVariable(varname='Latitude_1km', 
                                         datatype='f8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    lats_netcdf.long_name = 'latitude'
    lats_netcdf.units = 'degrees_north'
    lats_netcdf[:, :] = lat

with netCDF4.Dataset(sidecar, "a", format="NETCDF4") as rootgrp:
    lons_netcdf = rootgrp.createVariable(varname='Longitude_1km',
                                         datatype='f8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    lons_netcdf.long_name = 'longitude'
    lons_netcdf.units = 'degrees_east'
    lons_netcdf[:, :] = lon
    

with netCDF4.Dataset(sidecar, 'a', format="NETCDF4") as rootgrp:
    cover_netcdf = rootgrp.createVariable(varname='STARE_cover_1km', 
                                          datatype='u8', 
                                          dimensions=('l_1km'),
                                          chunksizes=[l_1km])
    cover_netcdf.long_name = 'SpatioTemporal Adaptive Resolution Encoding (STARE) cover'

with netCDF4.Dataset(sidecar, "a", format="NETCDF4") as rootgrp:
    sids_netcdf = rootgrp.createVariable(varname='STARE_index_1km', 
                                         datatype='u8', 
                                         dimensions=('i_1km', 'j_1km'),
                                         chunksizes=[i_1km, j_1km])
    sids_netcdf.long_name = 'SpatioTemporal Adaptive Resolution Encoding (STARE) index'
    sids_netcdf[:, :] = sids


# Use the modules

## Mod09

In [1]:
import importlib
import sys
sys.path.insert(0,'../') 

In [26]:
import products
importlib.reload(products)

<module 'products' from '../products/__init__.py'>

In [27]:
file_path = '/home/griessbaum/MOD09/MOD09.A2019317.0815.006.2019319020759.hdf'
granule = products.MOD09(file_path)

In [7]:
granule.read_gring()
granule.read_laton()

In [8]:
chunk_size = 500
lat_x = xarray.DataArray(lat, dims=['x', 'y']).chunk({'x': chunk_size})
lon_x = xarray.DataArray(lon, dims=['x', 'y']).chunk({'x': chunk_size})
with dask.distributed.Client(n_workers=4) as client:
    sids = xarray.apply_ufunc(pystare.from_latlon2D,
                              lat_x, 
                              lon_x,
                              kwargs={'adapt_resolution': True},
                              dask='parallelized',                          
                              output_dtypes=[numpy.int64])
sids = numpy.array(sids)

In [15]:
cover_res = int(pystare.spatial_resolution(sids).max())
cover_sids = pystare.to_nonconvex_hull_range_from_latlon(granule.gring_lats,
                                                         granule.gring_lats,
                                                         13)

## Sidecar

In [12]:
import sidecar
importlib.reload(sidecar)

<module 'sidecar' from '../sidecar.py'>

In [13]:
scar = sidecar.Sidecar(file_path)

In [16]:
i = sids.shape[0]
j = sids.shape[1]
l = cover_sids.size 
scar.write_dimensions(i, j, l)

In [21]:
granule.lons.shape[0]

2030

In [18]:
scar.write_lons(granule.lons, nom_res='1km')
scar.write_lats(granule.lats, nom_res='1km')
scar.write_sids(sids, nom_res='1km')
scar.write_cover(cover_sids, nom_res='1km')

ValueError: shape mismatch: objects cannot be broadcast to a single shape

# VNP02DNB

In [42]:
importlib.reload(products)

<module 'products' from '../products/__init__.py'>

In [43]:
file_path = '/home/griessbaum/MOD09/VNP03DNB.A2012092.1642.001.2017285224354.nc'
vnp02 = products.VNP03DNB(file_path)

In [46]:
vnp02.lats.astype(numpy.double)

array([[10.78973866, 10.78894711, 10.78815651, ...,  6.6385808 ,
         6.63725662,  6.63592243],
       [10.79673862, 10.79594517, 10.79515171, ...,  6.64541388,
         6.64408588,  6.64275074],
       [10.80373955, 10.80294228, 10.80214596, ...,  6.65226889,
         6.65090799,  6.64946222],
       ...,
       [31.55126953, 31.5510273 , 31.55078506, ..., 26.86467171,
        26.86257744, 26.86047745],
       [31.55830193, 31.55805588, 31.55781364, ..., 26.87133217,
        26.86923599, 26.86714172],
       [31.56533241, 31.56508827, 31.56484222, ..., 26.87798882,
        26.87590027, 26.87380409]])